In [4]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import SISE
import numpy as np
from sklearn.linear_model import Ridge
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt


In [5]:
# Function to preprocess image
def preprocess_image(image_path, target_size=(299, 299)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img[0]

# Function to generate semantic input samples
def semantic_input_sampling(image, sample_size=100):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    samples = np.array([datagen.random_transform(image) for _ in range(sample_size)])
    return samples

# Explain instance using SISE
def explain_instance_sise(image, model, sample_size=100):
    samples = semantic_input_sampling(image, sample_size)
    predictions = model.predict(samples)

    # Flatten the samples for the regression model
    feature_matrix = samples.reshape(samples.shape[0], -1)

    weights = np.exp(-np.sum((samples - image) ** 2, axis=(1, 2, 3)) / 0.1)
    surrogate_model = Ridge(alpha=1, fit_intercept=True)
    surrogate_model.fit(feature_matrix, predictions, sample_weight=weights)

    return surrogate_model.coef_

# Visualize explanation
def visualize_explanation(image, explanation, sample_size=100):
    samples = semantic_input_sampling(image, sample_size)
    top_features = np.argsort(np.abs(explanation))[-5:]
    for i, sample in enumerate(samples):
        if i in top_features:
            plt.imshow(sample)
            plt.axis('off')
            plt.show()


In [6]:
print ('REMEMBER TO UPDATE YOUR_PATH (where images, models are)!')

# This is the name of your model wrapper (InceptionV3 and GoogleNet are provided in model.py)
model_to_run = 'GoogleNet'  
user = 'yhkim'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_class_test'
working_dir = "./tmp/" + user + '/' + project_name
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir+ '/activations/'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
# where the images live.

# TODO: replace 'YOUR_PATH' with path to downloaded models and images. 
source_dir = './tcav/tcav_examples/image_models/imagenet/test_data'
bottlenecks = [ 'mixed4c', 'mixed4d', 'mixed4e']  # @param 
      
utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]

target = 'zebra'
concepts = ["dotted","striped","zigzagged"]

REMEMBER TO UPDATE YOUR_PATH (where images, models are)!


In [9]:
# Model load
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=True)
# Preprocess the image
image_path = "./test_data/zebra/1_zebra1.jpg"  # 로컬에 저장된 이미지 파일 경로
image = preprocess_image(image_path)

# Explain the image using SISE
# explanation = explain_instance_sise(image, model)

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-06-14 15:21:02.690747: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-14 15:21:02.690937: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


102967424/102967424 [==============================] - 4s 0us/step


In [ ]:
sise = SISE(model, '')